In [25]:
from pprint import pprint
import copy
import types
from jsonpath_ng.ext import parse
from abc import ABCMeta, abstractmethod
from pydantic import BaseModel, validator
from typing import List, Optional, Any, Union
import datetime

class Chainable:
    pass

#Todo: default params
#Todo: store secrets
class ChainableContext:
    def __init__(self, dryrun, debug):
        self.dryrun = dryrun
        self.debug = debug
        self.workflow = []

    def add_mapping(self, mapping):
        self.workflow.append(mapping)

    def run(self):
        return

class ChainableObjectHistory(BaseModel):
    func: Any#type(AbstractChainableFunction) #Union[ChainableFunction, TypeSafeChainableFunction]
    #data: Optional[Union[dict, TypeSafeChainableFunction.Data]] = None
    #param: Optional[Union[dict, TypeSafeChainableFunction.Param]] = None
    data: Optional[dict] = None
    param: Optional[dict] = None
    mapping: Optional[dict] = None
    
    @validator("func")
    def validate_some_foo(cls, val):
        if issubclass(type(val), AbstractChainableFunction):
            return val
        raise TypeError("Wrong type for 'func', must be subclass of AbstractChainableFunction")
    
class ChainableObject(BaseModel):
    data: Optional[dict] = {}
    meta: Optional[dict] = {}
    hist: Optional[List[ChainableObjectHistory]] = []
    mapping: Optional[dict] = {}

    def resolve(self, mapping):
        #make components availabel in function scobe
        data = self.data
        meta = self.meta
        #hist = self.hist
        for key in mapping['param']:
            res = []
            value = mapping['param'][key]
            if isinstance(value, dict): #dynamic
                if 'static' in value: res.append(value['static']) #explicit static 
                if 'eval' in value: res.append(eval(value['eval'])) #eval expression
                if 'jsonpath' in value:
                    jsonpath_expr = parse(value['jsonpath'])
                    for match in jsonpath_expr.find(self.dict()):
                        res.append(match.value)
                if 'match' in value:  #match condition
                    if 'meta' in value['match']:
                        jsonpath_expr = parse(value['match']['meta']['jsonpath'])
                        #[print(str(match.full_path)) for match in jsonpath_expr.find(self.content)]
                        #[pprint(match.value) for match in jsonpath_expr.find(obj.content)]
                        for match in jsonpath_expr.find(self.dict()):
                            data_path = str(match.full_path).replace('meta', 'data', 1) #default: replace only root key => traverse to data branch
                            if 'value' in value and 'data' in value['value']:
                                data_path = str(match.full_path).replace('meta', 'data', 1) 
                                if value['value']['data']['jsonpath'] != "": data_path += "." + value['value']['data']['jsonpath'] #value path relative to match path
                                res.append(parse(data_path).find(self.dict())[0].value)
                            if 'value' in value and 'meta' in value['value']:
                                data_path = str(match.full_path) + "." + value['value']['meta']['jsonpath'] #value path relative to match path
                                res.append(parse(data_path).find(self.dict())[0].value)
                            print(data_path)
                            #pprint(parse(data_path).find(self.content)[0].value)
                            
                if (len(res) == 1): res = res[0]
                mapping['param'][key] = res
            if isinstance(value, types.FunctionType): #dynamic function
                mapping['param'][key] = value()
        return mapping
    
    #todo async apply
    def apply(self, mapping):
        if mapping is None:
            mapping = self.mapping #read map from object (result from previous function)
        mapping = self.resolve(mapping)
        self = eval(mapping['func'] + ".apply(self, mapping['param'])")
        #raw_data(self.content, mapping)
        #pprint(mapping)
        #pprint(self.content)
        return self
    
    #Todo: consider file or db backends
    def store_data(self, key: str, data, meta):
        if not key in self.data: self.data[key] = []
        self.data[key].append(data)
        if not key in self.meta: self.meta[key] = []
        self.meta[key].append(meta)
        
    def store_hist(self, hist: ChainableObjectHistory):
        self.hist.append(hist)
        
class AbstractChainableFunction(BaseModel):
    name: str
    uuid: str
    start_time: Optional[datetime.datetime] = None
    end_time: Optional[datetime.datetime] = None
    obj: Optional[ChainableObject] = None
        
    @abstractmethod
    def apply(self, obj, param):
        pass   
    
class ChainableFunction(AbstractChainableFunction):
    param_default: dict = {}
    
    def set_default_params(self, param: dict):
        self.param_default = param 
    
    def set_param(self, param: dict = None):        
        if not param is None:
            param = {**self.param_default, **param}
        else: param = self.param_default
        pprint(param)
        return param
    
    def apply(self, obj: ChainableObject, param: dict = None):
        self.obj = obj
        param = self.set_param(param)
        self.pre_exec(param)
        self.func(param)
        self.post_exec(param)
        return obj
    
    def pre_exec(self, param):
        self.start_time = datetime.datetime.utcnow()
    
    @abstractmethod
    def func(self, param):
        pass
    
    def store(self, key, data, meta):
        self.obj.store_data(key, data, meta)
        
    def post_exec(self, param):
        obj = self.obj
        self.obj = None
        hist = {}
        #if param['debug']: hist['data'] = copy.deepcopy(obj.data)
        #if param['debug']: hist['mapping'] = copy.deepcopy(obj.mapping)
        self.end_time = datetime.datetime.utcnow()
        hist['func'] = self
        hist['param'] = param
        obj.store_hist(ChainableObjectHistory(**hist))
    
class TypeSafeChainableFunction(AbstractChainableFunction):
    class Param(BaseModel):
        debug: bool = False
    class Data(BaseModel):
        pass
    class Meta(BaseModel):
        data_class: Optional[type(BaseModel)] = None  
        data_class_name: Optional[str] = ""  
        
    param_class: type(Param)# = TypeSafeChainableFunction.Param  
    data_class: Optional[type(Data)]
    meta_class: Optional[type(Meta)]
    
    def __init__(self, name: str, uuid: str, 
                 param_class: type(Param) = Param,
                 data_class: type(Data) = None,
                 meta_class: type(Meta) = None
                ): 
        assert type(param_class) == type(TypeSafeChainableFunction.Param) or issubclass(param_class, TypeSafeChainableFunction.Param)
        assert data_class == None or type(data_class) == type(TypeSafeChainableFunction.Data) or issubclass(data_class, TypeSafeChainableFunction.Data)
        assert meta_class == None or type(meta_class) == type(TypeSafeChainableFunction.Meta) or issubclass(meta_class, TypeSafeChainableFunction.Meta)
        super().__init__(name=name, uuid=uuid, param_class=param_class, data_class=data_class, meta_class=meta_class)

    def apply(self, obj, param):
        self.obj = obj
        param = self.param_class(**param)
        self.pre_exec(param)
        self.func(param)
        self.post_exec(param)
        return obj
    
    def pre_exec(self, param):
        self.start_time = datetime.datetime.utcnow()
    
    @abstractmethod
    def func(self, param: Param):
        pass
    
    def store(self, key: str, data, meta):
        if self.data_class != None:
            if type(data) != self.data_class:
                data = self.data_class(**data)
        if self.meta_class != None:
            if type(meta) != self.meta_class:
                meta = self.meta_class(data_class = self.data_class, data_class_name = self.__class__.__name__ + '.' + self.data_class.__name__, **meta)
        self.obj.store_data(key, data, meta)
        
    def post_exec(self, param):      
        obj = self.obj
        self.obj = None
        hist = {}
        #if param['debug']: hist['data'] = copy.deepcopy(obj.data)
        #if param['debug']: hist['mapping'] = copy.deepcopy(obj.mapping)
        self.end_time = datetime.datetime.utcnow()
        hist['func'] = self.copy()
        hist['param'] = param
        obj.store_hist(ChainableObjectHistory(**hist))    
    
class GetRaw(ChainableFunction):
    def __init__(self):
        super().__init__(name="get_raw", uuid="0001")
        super().set_default_params({'debug': True, 'data_name':"raw"})

    def func(self, param):
        super().store('raw', [1,2,3,4], {'type': "list", 'dim': 1, 'name': param['data_name'], 'label': {'de':"Spannung"}, 'quant': "qudt:Voltage", 'unit':"qudt:mV", 'test':{'nested': "value"}})
    
class GetRawTypeSafe(TypeSafeChainableFunction):
    class Param(TypeSafeChainableFunction.Param):
        data_name: str = "raw"
        values: Dict[str, Union[str,dict]] {'key': {}, 'key2': "string"}
    
    def __init__(self):
        super(__class__, self).__init__(name="get_raw", uuid="0001", param_class=__class__.Param)

    def func(self, param):
        #param.debug
        super().store('raw', [1,2,3,4], {'type': "list", 'dim': 1, 'name': param.data_name, 'label': {'de':"Spannung"}, 'quant': "qudt:Voltage", 'unit':"qudt:mV", 'test':{'nested': "value"}}) 

class GetRawFullTypeSafe(TypeSafeChainableFunction):
    class Param(TypeSafeChainableFunction.Param):
        data_name: str = "raw"
        
    class Data(TypeSafeChainableFunction.Data):
        content: List[int]
        
    class Meta(TypeSafeChainableFunction.Meta):
        name: str
        label: dict[str, str]
        quant: str
    
    def __init__(self):
        super(__class__, self).__init__(name="get_raw", uuid="0001", param_class=__class__.Param, data_class=__class__.Data, meta_class=__class__.Meta)

    def func(self, param):
        super().store('raw', {'content':[1,2,3,4]}, {'type': "list", 'dim': 1, 'name': param.data_name, 'label': {'de':"Spannung"}, 'quant': "qudt:Voltage", 'unit':"qudt:mV", 'test':{'nested': "value"}}) 

class RemoveListElement(TypeSafeChainableFunction):
    class Param(TypeSafeChainableFunction.Param):
        l: List[Any]
    
    def __init__(self):
        super(RemoveListElement, self).__init__(name="remove_list_element", uuid="0003", param_class=RemoveListElement.Param)

    def func(self, param):
        print(type(param))
        print(param.l)
        del param.l[-1]
        print(param.l)
        
#m = TypeSafeChainableFunction.Meta()
#print(type(m))
#GetRaw.name
get_raw = GetRaw()
obj = ChainableObject()
obj = get_raw.apply(obj)
pprint(obj.dict())
get_raw2 = GetRawTypeSafe()
obj = ChainableObject()
obj = get_raw2.apply(obj, {'debug': True, 'data_name': "Test"})
pprint(obj.dict())
get_raw3 = GetRawFullTypeSafe()
obj = ChainableObject()
obj = get_raw3.apply(obj, {'debug': True, 'data_name': "Test"})
pprint(obj.dict())
#print(obj.meta['raw'][0].dict())
#pprint(obj.meta['raw'][0].data_class.schema())
#pprint(type(obj.meta['raw'][0].data_class))
mapping = { 'param': {
    'param3': {'match': {'meta': {'jsonpath': 'meta.*[?name = "Test"]'}}, 'value': {'data': {'jsonpath': 'content'}}}, #default: traverse to data branch
    'param4': {'match': {'meta': {'jsonpath': 'meta.*[?data_class_name = "GetRawFullTypeSafe.Data"]'}}, 'value': {'data': {'jsonpath': 'content'}}}, #value path relative to match path
}}
mapping = obj.resolve(mapping)
pprint(mapping)
        
l1 = lambda: False
obj = ChainableObject()
obj.apply({
    'func': "get_raw",
    'param': {'debug': False}
})
pprint(obj.dict())
mapping = { 'param': {
    'debug1': True,
    'debug2' : lambda: False,
    'debug3': {'static': False},
    #'debug4': {'eval': "hist[-1]['func']['name'] == 'get_raw'"},
    'param1': {'eval': "data['raw'][0]"},
    'param2': {'jsonpath': 'meta.*[?name = "raw"].label.de'}, #eval jsonpath
    'param3': {'match': {'meta': {'jsonpath': 'meta.*[?name = "raw"]'}}, 'value': {'data': {'jsonpath': '[0]'}}}, #default: traverse to data branch
    'param4': {'match': {'meta': {'jsonpath': 'meta.*[?name = "raw"]'}}, 'value': {'meta': {'jsonpath': 'label'}}}, #value path relative to match path
}}
mapping = obj.resolve(mapping)
pprint(mapping)
    
def get_mapping():
    return {
        'func': "Chainable.get_raw",
        'param': {'debug': {'static': False}}
    }
      
#Chainable.get_raw = GetRaw()
Chainable.get_raw = GetRawTypeSafe()
Chainable.remove_list_element = RemoveListElement()
obj = ChainableObject()
obj = obj.apply({
    'func': "Chainable.get_raw",
    'param': {'debug': False}
}).apply(get_mapping())
pprint(obj.dict())

workflow = [{
    'func': "Chainable.get_raw",
    'param': {'debug': False}
},{
    'func': "Chainable.get_raw",
    'param': {'debug': False, 'data_name': "RawVoltage"}
},{
    'func': "Chainable.remove_list_element",
    'param': {'l': {'match': {'meta': {'jsonpath': 'meta.*[?name = "RawVoltage"]'}}, 'value': {'data': {'jsonpath': ''}}}}
}]
obj2 = ChainableObject()
for step in workflow:
    obj2 = obj2.apply(step)
pprint(obj2.dict())



{'data_name': 'raw', 'debug': True}
{'data': {'raw': [[1, 2, 3, 4]]},
 'hist': [{'data': None,
           'func': {'end_time': datetime.datetime(2022, 5, 9, 5, 59, 58, 903901),
                    'name': 'get_raw',
                    'obj': None,
                    'param_default': {'data_name': 'raw', 'debug': True},
                    'start_time': datetime.datetime(2022, 5, 9, 5, 59, 58, 903863),
                    'uuid': '0001'},
           'mapping': None,
           'param': {'data_name': 'raw', 'debug': True}}],
 'mapping': {},
 'meta': {'raw': [{'dim': 1,
                   'label': {'de': 'Spannung'},
                   'name': 'raw',
                   'quant': 'qudt:Voltage',
                   'test': {'nested': 'value'},
                   'type': 'list',
                   'unit': 'qudt:mV'}]}}
{'data': {'raw': [[1, 2, 3, 4]]},
 'hist': [{'data': None,
           'func': {'data_class': None,
                    'end_time': datetime.datetime(2022, 5, 9, 5, 59, 58, 91

In [42]:
from abc import ABCMeta, abstractmethod
from pydantic import BaseModel
from typing import List, Optional
#from dataclass_wizard import JSONWizard
#from dataclass_wizard import fromdict, asdict
#from dataclasses import dataclass
#from pydantic.dataclasses import dataclass

class Data(BaseModel):
#@dataclass
#class Data(BaseModel, JSONWizard):
#@dataclass
#class Data(JSONWizard):
    id: int
    ks: str
    items: List[str]
    sub: Optional[Data] = None

data_dict = {'id': 1, 'ks': 'test', 'items': ['1', '2', '3'], 'sub': {'id': '1', 'ks': 'test', 'items': ['1']}}
data = Data(**data_dict)
#data = fromdict(Data, data_dict)
#print(data.id)
#print(data.sub.id)
print(repr(data))
data.sub.items

Data(id=1, ks='test', items=['1', '2', '3'], sub=Data(id=1, ks='test', items=['1'], sub=None))


In [223]:
class A():
    def __init__(self, a):
        self.a = 1

class B(BaseModel):
    b: Any        

class C(BaseModel):
    c: int
    
class D(BaseModel):
    l: List[Any]
    d: dict
        
b = B(b=A(a=1))
pprint(b.dict()['b'].a)

a = A(2)
b1 = B(b=a)

c1=C(c=1)
c2=C(c=2)
d = D(l=[c1, c2], d={'c1':c1, 'c2':c2})
pprint(d.dict())

1
{'d': {'c1': {'c': 1}, 'c2': {'c': 2}}, 'l': [{'c': 1}, {'c': 2}]}


In [ ]:
def func(x: int, y: int):
    x + y
@overload
def func(x: str, y: str)
    Int(x) 
    print(f"{x}...
    
class P(BaseType):
    x: Union[None, int, str]
    y: Union[None, int, str]
    
class PInt(P):
    x: int
    y: int
    
def func(p: P):
    try (PInt(p))
    pprint(p.dict())

    
    type(p.x) == int